In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from collections import defaultdict
import csv
from pyspark.sql import Row
from pyspark.ml.clustering import KMeans,KMeansModel
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import Word2Vec
from collections import Counter
import math
%load_ext autoreload
%autoreload 2

In [ ]:
def computeTF(wordDict, bow):
    tfDict = {}
    bowCount = len(bow)
    for word, count in wordDict.items():
        tfDict[word] = count/float(bowCount)
    return tfDict

def computeIDF(docList):
    idfDict = {}
    N = len(docList)    
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for doc in docList:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log10(N / float(val))
        
    return idfDict
def computeTFIDF(tfBow, idfs):
    all_TFIDF = []
    for tfBow in all_TF:
        tfidf = {}
        for word, val in tfBow.items():
            tfidf[word] = val*idfs[word]
        all_TFIDF.append(tfidf)
    return all_TFIDF

In [ ]:
episodes = defaultdict(list)
with open("/home/hadoop/Spark-Programs/csv/topics.csv", "r") as sentences_file:
    reader = csv.reader(sentences_file, delimiter=',')
    for row in reader:
        episodes[row[0]] = (','.join(row[1:]))  
        
counted_topics_set = []
all_topics_sets = {}
for topics in episodes.values():
    all_topics_sets = set(topics_sets).union(set(topics.split(',')))
    counted_topics = Counter(topics.split(','))
    counted_topics_set.append(dict(counted_topics)) 
intial_vectors = []
all_TF = []
for files in counted_topics_set:
    intial_vector = dict.fromkeys(all_topics_sets, 0)
    for topic in files:
        intial_vector[topic]+=files[topic]
    TF = computeTF(intial_vector,files.keys())
    intial_vectors.append(intial_vector)
    all_TF.append(TF)

IDFs = computeIDF(intial_vectors)

TFIDFs = computeTFIDF(all_TF,IDFs)

topics_Dict = (pd.DataFrame(TFIDFs))
np.savetxt('/home/hadoop/output.txt', topics_Dict.values, delimiter=",", fmt="%s") 






In [ ]:
def f(x):
    rel = {}
    print(type(x))
    rel['features'] = Vectors.dense([float(a) for a in x])
    return rel 

df = sc.textFile("/home/hadoop/output").map(lambda line: line.split(','))

df = df.map(lambda p: Row(**f(p))).toDF()

kmeansmodel = KMeans().setK(6).setFeaturesCol('features')
kmeansmodel = kmeansmodel.setPredictionCol('prediction')
kmeansmodel = kmeansmodel.fit(df)
results = kmeansmodel.transform(df).collect()
counter = 1
for item in results:
    print('No.'+ str(counter) +' data'+' is predcted as cluster'+ str(item[1]))
    counter+=1
    
    